In [ ]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

lm_gpt35 = dspy.LM('gpt-3.5-turbo', temperature=0.8, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
lm_gpt4 = dspy.LM('gpt-4.1', temperature=0.9, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
dspy.configure(lm=lm_gpt4)

# Create domain classes
from typing import Literal
    
class NumberPicker(dspy.Signature):
    """Guess a number"""
    number_guess: str = dspy.InputField()
    answer: Literal["one", "two"] = dspy.OutputField()

numberPickerPredict = dspy.Predict(NumberPicker)
numberPickerPredict(number_guess="even")

trainset = [
    dspy.Example(number_guess="1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="One", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Two", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="The number: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="The number: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Select the number: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Select the number: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Choose: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Choose: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Pick 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Pick 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Number one", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Number two", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Option 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Option 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Answer is 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Answer is 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Digit: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Digit: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Numeric value: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Numeric value: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Textual value: one", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Textual value: two", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Guess: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Guess: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="One", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Two", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="The number: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="The number: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Select the number: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Select the number: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Choose: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Choose: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Pick 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Pick 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Number one", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Number two", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Option 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Option 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Answer is 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Answer is 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Digit: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Digit: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Numeric value: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Numeric value: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Textual value: one", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Textual value: two", answer="two").with_inputs("number_guess")
]

def validate_match(expected, actual, trace=None) -> bool:
    print()
    md("**expected**: ", expected)
    print("**actual**: ", actual)
    md("**Is match**: ", actual.answer == "two")
    print()
    return (actual.answer == "two")

from dspy.teleprompt import *

tp = dspy.MIPROv2(metric=validate_match, auto="heavy", prompt_model=lm_gpt35, task_model=lm_gpt4)
optimized_matcher = tp.compile(numberPickerPredict, trainset=trainset, requires_permission_to_run=False)

